This project will show how adaptable neural networks can be, we will now attemp to use a neural network to learn the optimal move for Tic-Tac-Toe. We will approach this knowing that Tic-Tac-Toe is a deterministic game and that the optimal moves are already known.

To train our model, we will use a list of board positions followed by the optimal response for a number of different boards. We can reduce the amount of boards to train on by considering only board position that are differnt with regard to symmetry. The non-identity transformations of a Tic-Tac-Toe board are a rotation (in either direction ) of 90 degress, 180 degrees and 270 degress a horizontal reflection and a vertical reflection. Given this idea, we will use a shortlist of boards with optimal move, apply 2 random transformations, and then feed that into our neural network for learning. 

Since Tic-Tac-Toe is a deterministic game, it is worth noting that whoever goes first should either win or draw. We will hope a model that can respond to our moves optimally and ultimately result in a raw.

To check how our model is performing, we will do 2 things. The first check we will perform is to remove a position and an optimal move row from our training set. This will allow us to see if the neural network can generalize a move it has not seen before. The second way to evaluate our model is to actually play a game agasint it at the end

https://github.com/lahoangquy/Tic-tac-toe-with-Neural-Network-YouTube

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Colab Notebooks

/content/drive/My Drive/Colab Notebooks


In [3]:
import tensorflow as tf
#import matplotlib.pyplot as plt
import csv
import numpy as np
import random

In [4]:
# Definition of X's, O's, and empty spots:
# X = 1
# O = -1
# empty = 0
# response on 1-9 grid for placement of next '1'

# For example, the 'test_board' is:
#
#   O  |  -  |  -
# -----------------
#   X  |  O  |  O
# -----------------
#   -  |  -  |  X
#
# board above = [-1, 0, 0, 1, -1, -1, 0, 0, 1]
# Optimal response would be position 6, where
# the position numbers are:
#
#   0  |  1  |  2
# -----------------
#   3  |  4  |  5
# -----------------
#   6  |  7  |  8


# Test board optimal response:
#response = 6
# Set batch size and five different symmetries of board positions

In [5]:
batch_size = 50


To make visualizng the boards a bit easier, we will need to create a function taht outputs Tic-Tac-Toe boards with Xs and Os

In [6]:
# Print a board
def print_board(board):
    symbols = ['O', ' ', 'X']
    board_plus1 = [int(x) + 1 for x in board]
    board_line1 = ' {} | {} | {}'.format(symbols[board_plus1[0]],
                                         symbols[board_plus1[1]],
                                         symbols[board_plus1[2]])
    board_line2 = ' {} | {} | {}'.format(symbols[board_plus1[3]],
                                         symbols[board_plus1[4]],
                                         symbols[board_plus1[5]])
    board_line3 = ' {} | {} | {}'.format(symbols[board_plus1[6]],
                                         symbols[board_plus1[7]],
                                         symbols[board_plus1[8]])
    print(board_line1)
    print('___________')
    print(board_line2)
    print('___________')
    print(board_line3)

Now we will need to create a function that will return a new board and an optimal response position under a transformation

In [7]:
# Data Augmentation
symmetry = ['rotate180', 'rotate90', 'rotate270', 'flip_v', 'flip_h']

# Given a board, a response, and a transformation, get the new board+response
def get_symmetry(board, play_response, transformation):
    """
    :param board: list of integers 9 long:
     opposing mark = -1
     friendly mark = 1
     empty space = 0
    :param play_response: integer of where response is (0-8)
    :param transformation: one of five transformations on a board:
     'rotate180', 'rotate90', 'rotate270', 'flip_v', 'flip_h'
    :return: tuple: (new_board, new_response)
    """
    if transformation == 'rotate180':
        new_response = 8 - play_response
        return board[::-1], new_response
    elif transformation == 'rotate90':
        new_response = [6, 3, 0, 7, 4, 1, 8, 5, 2].index(play_response)
        tuple_board = list(zip(*[board[6:9], board[3:6], board[0:3]]))
        return [value for item in tuple_board for value in item], new_response
    elif transformation == 'rotate270':
        new_response = [2, 5, 8, 1, 4, 7, 0, 3, 6].index(play_response)
        tuple_board = list(zip(*[board[0:3], board[3:6], board[6:9]]))[::-1]
        return [value for item in tuple_board for value in item], new_response
    elif transformation == 'flip_v':
        new_response = [6, 7, 8, 3, 4, 5, 0, 1, 2].index(play_response)
        return board[6:9] + board[3:6] + board[0:3], new_response
    elif transformation == 'flip_h':  # flip_h = rotate180, then flip_v
        new_response = [2, 1, 0, 5, 4, 3, 8, 7, 6].index(play_response)
        new_board = board[::-1]
        return new_board[6:9] + new_board[3:6] + new_board[0:3], new_response
    else:
        raise ValueError('Method not implemented.')

We will create a function that will load the file with the boards and responses and will store it as a list of tuples

In [8]:
def get_moves_from_csv(csv_file):
    """
    :param csv_file: csv file location containing the boards w/ responses
    :return: moves: list of moves with index of best response
    """
    play_moves = []
    with open(csv_file, 'rt') as csvfile:
        reader = csv.reader(csvfile, delimiter=',')
        for row in reader:
            play_moves.append(([int(x) for x in row[0:9]], int(row[9])))
    return play_moves

Now we do need to tie everything together to create a function that will return a randomly transformed board and response

In [9]:
def get_rand_move(play_moves, rand_transforms=2):
    """
    :param play_moves: list of the boards w/responses
    :param rand_transforms: how many random transforms performed on each
    :return: (board, response), board is a list of 9 integers, response is 1 int
    """
    (board, play_response) = random.choice(play_moves)
    possible_transforms = ['rotate90', 'rotate180', 'rotate270', 'flip_v', 'flip_h']
    for _ in range(rand_transforms):
        random_transform = random.choice(possible_transforms)
        (board, play_response) = get_symmetry(board, play_response, random_transform)
    return board, play_response

Now we will load our data and create a training set

In [10]:
moves = get_moves_from_csv('base_tic_tac_toe_moves.csv')
# Create a train set:
train_length = 500
train_set = []
for t in range(train_length):
    train_set.append(get_rand_move(moves))

Rememer that we want to remove one board and an optimal response from our training set to see if the model can generalize making the best move.

In [11]:
# To see if the network learns anything new, we will remove
# all instances of the board [-1, 0, 0, 1, -1, -1, 0, 0, 1],
# which the optimal response will be the index '6'.  We will
# Test this at the end.
test_board = [-1, 0, 0, 1, -1, -1, 0, 0, 1]
train_set = [x for x in train_set if x[0] != test_board]

In [12]:
# We do need to initialize the weights and bias and create our models
def init_weights(shape):
    return tf.Variable(tf.random.normal(shape))
A1 = init_weights([9, 81])
bias1 = init_weights([81])
A2 = init_weights([81, 9])
bias2 = init_weights([9])

Now we will create our mode. Note that we do not include the softmax() activation function because it is included in the lost function

In [13]:
# Initialize input data
X = tf.keras.Input(dtype=tf.float32, batch_input_shape=[None, 9])
hidden_output = tf.keras.layers.Lambda(lambda x: tf.nn.sigmoid(tf.add(tf.matmul(x, A1), bias1)))(X)
# Note: we don't take the softmax at the end because our cost function does that for us
final_output = tf.keras.layers.Lambda(lambda x: tf.add(tf.matmul(x, A2), bias2))(hidden_output)
model = tf.keras.Model(inputs=X, outputs=final_output, name="tic tac toe")

The following Variables were used a Lambda layer's call (lambda), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(9, 81) dtype=float32>
  <tf.Variable 'Variable:0' shape=(81,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.
The following Variables were used a Lambda layer's call (lambda_1), but
are not present in its tracked objects:
  <tf.Variable 'Variable:0' shape=(81, 9) dtype=float32>
  <tf.Variable 'Variable:0' shape=(9,) dtype=float32>
It is possible that this is intended behavior, but it is more likely
an omission. This is a strong indication that this layer should be
formulated as a subclassed Layer rather than a Lambda layer.


In [14]:
# We will declare our optimizer
optimizer = tf.keras.optimizers.SGD(0.025)

We will now loop through the training of our neural network. Note that our loss function will be the average softmax of the final output logits

In [15]:
loss_vec = []
for i in range(10000):
    rand_indices = np.random.choice(range(len(train_set)), batch_size, replace=False)
    batch_data = [train_set[i] for i in rand_indices]
    x_input = [x[0] for x in batch_data]
    y_target = np.array([y[1] for y in batch_data])

    # Open a GradientTape.
    with tf.GradientTape(persistent=True) as tape:

        # Forward pass.
        output = model(np.array(x_input, dtype=float))

        # Apply loss function (Cross Entropy loss)
        loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=y_target))
        loss_vec.append(loss)

    # Get gradients of loss with reference to the weights and bias variables to adjust.
    gradients_A1 = tape.gradient(loss, A1)
    gradients_b1 = tape.gradient(loss, bias1)
    gradients_A2 = tape.gradient(loss, A2)
    gradients_b2 = tape.gradient(loss, bias2)

    # Update the weights and bias variables of the model.
    optimizer.apply_gradients(zip([gradients_A1, gradients_b1, gradients_A2, gradients_b2],
                                  [A1, bias1, A2, bias2]))

    if i % 500 == 0:
        print('Iteration: {}, Loss: {}'.format(i, loss))

Iteration: 0, Loss: 7.945645332336426
Iteration: 500, Loss: 1.865727424621582
Iteration: 1000, Loss: 1.5998142957687378
Iteration: 1500, Loss: 1.6589170694351196
Iteration: 2000, Loss: 1.3871594667434692
Iteration: 2500, Loss: 1.3872671127319336
Iteration: 3000, Loss: 1.1046247482299805
Iteration: 3500, Loss: 1.0914956331253052
Iteration: 4000, Loss: 1.048866868019104
Iteration: 4500, Loss: 0.8853231072425842
Iteration: 5000, Loss: 0.9256417155265808
Iteration: 5500, Loss: 0.9089015126228333
Iteration: 6000, Loss: 0.9707467555999756
Iteration: 6500, Loss: 0.81446373462677
Iteration: 7000, Loss: 0.9326468110084534
Iteration: 7500, Loss: 0.6198672652244568
Iteration: 8000, Loss: 0.6715857982635498
Iteration: 8500, Loss: 0.7007814049720764
Iteration: 9000, Loss: 0.8517657518386841
Iteration: 9500, Loss: 0.682719886302948
